# 总体架构

&emsp;&emsp;&emsp;&emsp;早期的机器阅读理解模型大多基于检索技术，但是信息检索主要依赖关键词的匹配，而在很多情况下，单纯依靠问题和文章片段的文字匹配找到的答案与问题并不相关。随着深度学习的发展，机器阅读理解步入了神经网络时代。本文将介绍基于深度学习的机器阅读理解模型的架构，探究其提升性能的原因。

&emsp;&emsp;&emsp;&emsp;基于深度学习的MRC模型构造各异，但是经过多年的实践和探索，逐渐形成了稳定框架结构。本节将介绍MRC模型共有的总体架构。

&emsp;&emsp;&emsp;&emsp;MRC模型的输入为文章和问题。因此，首先要对这两部分进行数字化编码，将其变成可以被计算机处理的信息单元。在编码的过程中，模型需要保留原有语句在文章中的语义。因此，每个单词、短语和句子的编码必须建立在理解上下文的基础上。我们把模型中进行编码的模块称为编码层。

&emsp;&emsp;&emsp;&emsp;接下来，由于文章和问题之间存在相关性，所以模型需要建立文章和问题之间的联系。例如，如果问题中出现关键词“河流”，而文章中出现关键词“长江”。虽然两个词不完全一样，但是其语义编码接近。因此，文章中“长江”一词以及邻近的语句将成为模型回答问题时的重点关注对象。这可以通过注意力机制加以解决。在这个过程中，MRC模型将文章和问题的语义结合在一起进行考量，进一步加深模型对于两者的理解。我们将这个模块称为交互层。交互层可以让模型聚焦文章和问题的语义联系，借助于文章的语义分析加深对问题的理解，同时借助于问题的语义分析加深对文章的理解。

&emsp;&emsp;&emsp;&emsp;经过交互层，模型建立起文章和问题之间的语义联系，接下来就可以预测问题的答案了。完成预测功能的模块称为输出层。MRC任务的答案有多种类型，因此输出层的具体形式需要和任务的答案类型相关联。此外，输出层确定模型优化时的评估函数和损失函数。

# 编码层

&emsp;&emsp;&emsp;&emsp;与其他基于深度学习的自然语言处理模型类似，MRC模型首先需要将文字形式的文章和问题转化成词向量。编码层一般采用类似的分布式算法对文本进行分词和向量化处理，然后加入字符编码等更丰富的信息，并采用上下文编码获得每个单词在具体语境中的含义。

## 词表的建立与初始化

&emsp;&emsp;&emsp;&emsp;首先，模型对训练文本进行分词以得到其中所有的单词。然后，根据阈值选取出现次数超过一定次数的单词组成词表，词表以外的单词视为非词表词（Out-Of-Vocabulary, OOV），用特殊单词<UNK\>表示。这样，模型可以得到一个大小为|V|的词表，每个词表中的词用一个d维向量表示。接下来，我们有两种方法获得词表中的单词向量：
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1、保持词表向量不变，即采用预训练词表中的向量（如Word2Vec的300维向量），在训练过程中不进行改变；
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2、将词表中的向量视为参数，在训练过程中和其他参数一起求导并优化。这里，既可以使用预训练词表向量进行初始化，也可以选择随机初始化。

&emsp;&emsp;&emsp;&emsp;第一种选择的优势是模型参数少，训练初期收敛较快；第二种选择的优势是可以根据实际数据调整词向量的值，以达到更好的训练效果。而采用预训练词表向量初始化一般可以使得模型在优化的最初几轮获得明显比随机初始化方法更优的结果。

&emsp;&emsp;&emsp;&emsp;在编码层中，为了更准确地表示每个单词在语句中的语义，除了词向量外，还经常对命名实体（named entity）和词性（part-of-speech）进行向量化处理。如果命名实体共有N种，则建立大小为N的命名实体表，每种命名实体用一个长度为d_N的向量表示；如果词性共有P种，则建立大小为P的词性表，每种词性用一个长度为d_P的向量表示。两个表中的向量均为可训练的参数。然后，用文本分析包，如spaCy，获得文章和问题中的每个词的命名实体和词性，再将对应向量拼接在词向量之后。由于一个词的命名实体属性与词性和这个词所在的语句有关，因此用这种方式获得向量编码可以更好地表示单词的语义，在许多模型中性能都有明显的提升。

&emsp;&emsp;&emsp;&emsp;另一种在机器阅读理解中非常有效的单词编码是精确匹配（exact matching）编码。exact matching编码适用于文章中的单词。对于文章中的单词w，检查w是否出现在问题中：如果是，w的精确匹配编码为1，否则为0，然后将这个二进制位拼接在单词向量后。精确匹配编码可以使模型快速找到文章中出现了问题单词的部分，而许多问题的答案往往就在这部分内容的附近。

## 字符编码

&emsp;&emsp;&emsp;&emsp;文本处理中时常会出现拼写错误，这时通过字符组合，往往可以识别正确的单词形式。此外，许多语言中存在词根的概念，即词的一部分是一个常见且有固定含义的子词。在单词的理解中，字符和子词具有很强的辅助作用。

&emsp;&emsp;&emsp;&emsp;为了更好地利用字符信息，在编码层中采用字符编码，即每个字符用一个向量表示。但是，由于单词的长度不一，每个单词可能有不同个数的字符向量。为了产生一个固定长度的字符信息向量，我们可以将多个字符向量合并成一个编码。最常用的模型是字符CNN。

&emsp;&emsp;&emsp;&emsp;设一个单词有K个字符，且对应的K个字符向量为（c_1, c_2, ..., c_k），每个向量维度为c。字符CNN利用一个窗口大小为W且有f个输出通道的CNN获得（K-W+1）个f维向量。然后，采用最大池化方法求得这些向量每一个维度上的最大值，形成一个f维向量作为结果。

In [2]:
# 字符卷积神经网络Char-CNN的PyTorch实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [40]:
class CharCNNMaxpooling(nn.Module):
    # char_num为字符表大小，char_dim为字符向量长度，window_size为CNN窗口长度，out_channels为CNN输出通道数
    def __init__(self, char_num, char_dim, window_size, out_channels):
        super(CharCNNMaxpooling, self).__init__()
        # 字符表向量，共有char_num个向量，每个维度为char_dim
        self.char_embed = nn.Embedding(num_embeddings=char_num, embedding_dim=char_dim)
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size*char_dim
        self.cnn = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size, char_dim))

    # 输入char_ids为batch组文本，每个文本长度为seq_len，每个词含word_len个字符编号（0~char_num-1），输入维度为batch*seq_len*word_len
    # 输出res为所有单词的字符向量表示，维度是batch*seq_len*out_channels
    def forward(self, char_ids):
        # 根据字符编号得到字符向量，结果维度为batch*seq_len*word_len*char_dim
        x = self.char_embed(char_ids)
        # 合并前两维并变成单通道，结果维度（batch*seq_len）*1*word_len*char_dim
        x_unsqueeze = x.view(-1, x.shape[2], x.shape[3]).unsqueeze(1)
        # CNN，结果维度为（batch*seq_len）*out_channels*new_seq_len*1
        x_cnn = self.cnn(x_unsqueeze)
        # 删除最后一维，结果维度为（batch*seq）*out_channels*new_seq_len
        x_cnn_result = x_cnn.squeeze(3)
        # 最大池化，遍历最后一维求最大值，结果维度为（batch*seq_len）*out_channels
        res, _ = x_cnn_result.max(2)  # 
        # print('res_如下:')
        # print(res)
        # print('res_: {shape}'.format(shape=res.shape))
        return res.view(x.shape[0], x.shape[1], -1)  # x.shape[0] = batch, x.shape[1] = seq_len，不确定第2个维度是几，但是一定指定前两个维度的大小，所以这里用-1代替

In [41]:
# test

In [42]:
batch = 10  # batch
seq_len = 20  # 每个文本长度
word_len = 12  # 单词中字符个数
char_num = 26  # 字符表大小
char_dim = 10  # 字符向量长度
window_size = 3
out_channels = 8
char_cnn = CharCNNMaxpooling(char_num=char_num, char_dim=char_dim, window_size=window_size, out_channels=out_channels)
char_ids = torch.LongTensor(batch, seq_len, word_len).random_(0, char_num - 1)
print('char_ids: {shape}'.format(shape=char_ids.shape))
res = char_cnn(char_ids)
print('res: {shape}'.format(shape=res.shape))
print(res)

char_ids: torch.Size([10, 20, 12])
res: torch.Size([10, 20, 8])
tensor([[[ 0.6529,  0.8465,  0.4470,  ...,  1.4186,  0.8840,  0.5334],
         [ 0.3931,  1.1519,  0.6541,  ...,  0.8053,  1.0912,  0.9075],
         [ 1.1718,  1.2721,  0.8915,  ...,  1.2315,  0.8225,  0.5993],
         ...,
         [ 0.3707,  0.8362,  0.3265,  ...,  1.7615,  1.3467,  1.0405],
         [ 0.9671,  1.0019,  0.5029,  ...,  1.5247,  1.1616,  1.0269],
         [ 0.3161,  1.0389,  0.9601,  ...,  1.4771,  1.2479,  1.4900]],

        [[ 0.4622,  1.3565,  0.9146,  ...,  1.8216,  1.0728,  0.7849],
         [ 1.4459,  0.7593,  0.7322,  ...,  0.7574,  1.2987,  1.0229],
         [ 0.8910,  0.9636,  0.5123,  ...,  0.7922,  0.6371,  0.8367],
         ...,
         [ 1.5457,  1.1434,  0.7236,  ...,  1.3477,  0.6684,  1.3263],
         [ 0.3809,  1.1418,  0.3265,  ...,  0.7690,  1.0942,  0.7274],
         [ 0.8428,  0.8657,  0.9694,  ...,  1.2150,  1.0022,  1.0709]],

        [[ 0.7381,  0.8110,  0.8606,  ...,  1.7145, 

In [12]:
char_cnn

CharCNNMaxpooling(
  (char_embed): Embedding(26, 10)
  (cnn): Conv2d(1, 8, kernel_size=(3, 10), stride=(1, 1))
)

## 上下文编码

&emsp;&emsp;&emsp;&emsp;理解一个单词需要考虑它的上下文，很多具有多义性的单词需要考虑它周围的语句才能确定其明确含义。然而，基于词表的单词向量是固定的，不会随上下文变化。这就会导致同一个词在不同的语句中语义不同但其向量表示完全相同的情况。因此，编码层需要对每个单词生成上下文编码（contextual embedding）。这种编码会随着单词的上下文不同而发生改变，从而反映单词在当前语句中的含义。

&emsp;&emsp;&emsp;&emsp;在深度学习中，为了实现上下文语义的理解通常采用单词之间的信息传递。RNN是最常用的上下文编码生成结构，因为RNN利用相邻单词间的状态向量转移实现语义信息的传递。为了更有效地利用每个单词左右两个方向的语句信息，常采用双向循环神经网络（bidirectional RNN）获得上下文编码。而许多模型还采用了多层RNN，用于提取更高级的上下文语义，获得更好的效果。以下是多层双向RNN获得文本单词上下文编码的代码。

In [43]:
import torch
import torch.nn as nn

In [44]:
class ContextualEmbedding(nn.Module):
    # word_dim为词向量维度，state_dim为RNN状态维度，rnn_layer为RNN层数
    def __init__(self, word_dim, state_dim, rnn_layer):
        super(ContextualEmbedding, self).__init__()
        # 多层双向GRU
        self.rnn = nn.GRU(input_size=word_dim, hidden_size=state_dim, num_layers=rnn_layer, bidirectional=True, batch_first=True)
    
    # 输入x为batch组文本，每个文本长度为seq_len，每个词用一个word_dim维向量表示，输入维度为batch*seq_len*word_dim
    # 输出res为所有单词的上下文向量表示，维度是batch*seq_len*out_dim
    def forward(self, x):
        res, _ = self.rnn(x)  # 输出维度是batch*seq_len*out_dim，其中out_dim=2*state_dim，包含两个方向
        return res

In [45]:
# test

In [46]:
batch = 10
seq_len = 20
word_dim = 50
state_dim = 100
rnn_layer = 2
x = torch.randn(batch, seq_len, word_dim)
context_embed = ContextualEmbedding(word_dim=word_dim, state_dim=state_dim, rnn_layer=rnn_layer)
res = context_embed(x)
print(res.shape)
print(res)

torch.Size([10, 20, 200])
tensor([[[ 0.0080,  0.0752,  0.0691,  ..., -0.1804, -0.1286, -0.0760],
         [ 0.0345,  0.2315, -0.0525,  ..., -0.0794, -0.0285, -0.0648],
         [-0.0214,  0.2942,  0.0620,  ...,  0.0956,  0.0647, -0.1344],
         ...,
         [ 0.0417,  0.2764, -0.0351,  ...,  0.0393, -0.0262, -0.0384],
         [-0.0468,  0.1864, -0.0866,  ...,  0.1061, -0.0113, -0.0976],
         [ 0.0095,  0.1128, -0.2422,  ...,  0.0659, -0.0028, -0.0171]],

        [[ 0.0631,  0.0126, -0.0295,  ..., -0.0325,  0.1695, -0.0288],
         [ 0.0397, -0.0197, -0.1112,  ...,  0.0062,  0.0411, -0.0849],
         [-0.0527, -0.0169, -0.1902,  ..., -0.0864, -0.1091, -0.0711],
         ...,
         [ 0.0062,  0.1072,  0.0422,  ...,  0.0799, -0.4598, -0.2337],
         [-0.0610,  0.2202,  0.0625,  ...,  0.1482, -0.3998, -0.0943],
         [-0.1604,  0.2246,  0.0001,  ...,  0.0522, -0.2840, -0.0291]],

        [[-0.0154,  0.1051,  0.2525,  ..., -0.1313,  0.0892, -0.1459],
         [-0.0266, 

&emsp;&emsp;&emsp;&emsp;研究者进一步发现，在大规模自然语言处理任务上进行预训练，然后将预训练模型中的循环神经网络参数用用机器阅读理解，可以获得明显的性能提升，如CoVe模型在大量机器翻译数据上训练序列到序列模型，然后将编码器部分的循环神经网络用于SQuAD数据集，F1分数提高了近4%。

&emsp;&emsp;&emsp;&emsp;综上所述，在编码层中，每个问题单词由词表向量、命名实体向量、词性向量、字符编码、上下文编码组成，而每个文章单词除了以上5中向量外，还有精确匹配编码。

# 交互层

&emsp;&emsp;&emsp;&emsp;机器阅读理解模型在编码层中获得了文章和问题中单词的语义向量表示，但两部分的编码是基本独立的。为了获得最终答案，模型需要交互处理文章和问题中的信息。因此，模型在交互层中将文章和问题的语义信息融合，基于对问题的理解分析文章，并基于对文章的理解分析问题，从而达到更深层次的理解。

## 互注意力

&emsp;&emsp;&emsp;&emsp;交互层的输入是编码层的输出，即文章的单词向量(p1, p2, ..., pn)和问题的单词向量(q1, q2, ..., qn)。为了对两部分的语义进行交互处理，一般采用注意力机制。注意力机制最初应用在序列到序列模型中，它的输入包括一个单词向量x和一个单词向量组A=(a1, a2, .., an)。注意力机制从向量x的角度对A进行总结，从而获得A所代表的语句和单词x相关的部分的信息。注意力机制的结果为向量x^A，是向量组A的线性组合。其中，与x相关的A中的单词获得相对较大的权重。例如，如果x为“踢球”的向量表示，A为“我|喜欢|足球”的向量表示(a1, a2, a3)，则注意力机制的结果较大可能类似为x^A=0.1a1+0.05a2+0.85a3。

&emsp;&emsp;&emsp;&emsp;在MRC中，可以用注意力机制计算从文章到问题的注意力向量：基于对文章第i个词pi的理解，对问题单词的向量组(q1, q2, ..., qn)的语义总结，得到一个向量p_i^q。注意力机制通过注意力函数对向量组的所有向量打分。一般而言，注意力函数需要反映pi和每个向量组qj的相似度。获得每个qj的分数后，使用softmax函数进行归一化，得到权重，最后使用得到的权重计算(q1, q2, ..., qn)的加权和，即为注意力机制的结果。值得注意的是，交互注意力的结果向量个数是文章单词的个数m，而维度是问题单词的编码长度|q1|。每个注意力向量p_i^q都是问题单词编码的线性组合，而系数来自文章和问题的语义关系相似度。以最简单的内积注意力函数为例，由于两个语义相近的词向量方向更为一致，所以注意力机制给予和文章单词i语义更相近的问题单词j更大的权重，从而达到交互信息的目的。利用类似方法也可以获得从问题到文章的注意力向量，这可以使模型在理解每个问题单词语义的同时兼顾对文章的理解。上述两种方式的注意力机制统称为互注意力。

## 自注意力

&emsp;&emsp;&emsp;&emsp;在计算上下文编码时，RNN以线性方式传递单词信息。在这个过程中，一个单词的信息随距离的增加而衰减，特别是当文章较长时，靠前部分的语句和靠后部分的语句几乎没有进行有效的状态传递。但是在一些文章中，要获得答案可能需要理解文章中若干段相隔较远的部分。为了解决这个问题，可以使用自注意力机制。自注意力计算一个向量组和自身的注意力向量Self-Attention。以下是自注意力计算的代码示例，其中使用参数矩阵W将原向量映射到隐藏层，然后计算内积得到注意力分数。这样做的好处是可以在向量维度较大时通过控制隐藏层大小降低时空复杂度。

In [2]:
import torch
import torch.nn as nn

In [7]:
class SelfAttention(nn.Module):
    def __init__(self, dim, hidden_dim):  # # dim为向量维度，hidden_dim为自注意力计算的隐藏层维度
        super(SelfAttention, self).__init__()
        self.W = nn.Linear(in_features=dim, out_features=hidden_dim)  # 参数矩阵W

    # x: 进行自注意力计算的向量组，维度为batch*n*dim
    def forward(self, x):
        hidden = self.W(x)  # 计算隐藏层，维度为batch*n*hidden_dim
        scores = hidden.bmm(hidden.transpose(1, 2))  # 注意力分数scores，维度为batch*n*n
        alpha = F.softmax(input=scores, dim=-1)  # 对最后一维进行softmax
        attended = alpha.bmm(x)  # 注意力向量，结果维度为batch*n*dim
        return attended

In [8]:
# test

In [9]:
batch = 10
n = 15
dim = 40
hidden_dim = 20
x = torch.randn(batch, n, dim)
self_attention = SelfAttention(dim=dim, hidden_dim=hidden_dim)
res = self_attention(x)
print(res.shape)
print(res)

torch.Size([10, 15, 40])
tensor([[[ 1.0636,  0.6413,  1.0036,  ...,  0.4912, -1.2613,  0.0011],
         [-0.6179, -0.1356,  0.4855,  ...,  0.0074,  0.0124,  0.1811],
         [ 0.2862, -0.0210, -0.6258,  ...,  0.0855, -0.1832,  1.9533],
         ...,
         [ 0.6672, -0.5016, -0.2981,  ...,  0.7955, -0.3975,  0.1764],
         [ 0.2800, -0.1291, -0.7119,  ...,  1.3739,  0.4695, -1.3520],
         [ 0.9948, -0.2858, -0.1074,  ...,  0.4295, -0.5246,  1.4150]],

        [[-0.7645,  0.6810, -0.0987,  ..., -0.5691, -0.6215, -0.6135],
         [-0.5088, -0.3746, -0.4264,  ..., -0.4812,  0.3636, -1.6231],
         [-0.0587,  0.2572, -0.4895,  ...,  1.3402, -0.4772,  0.4431],
         ...,
         [-2.0054,  0.1587,  0.8061,  ...,  0.6684, -0.0413, -0.8363],
         [ 0.0734,  1.2391, -0.4425,  ..., -1.1353,  0.9222,  0.2684],
         [ 0.7585,  0.1512, -1.5114,  ...,  0.3267,  0.5089, -1.0395]],

        [[-1.2238, -0.3011, -0.8977,  ...,  0.4832,  0.6103,  0.8707],
         [-1.0241, -

&emsp;&emsp;&emsp;&emsp;在自注意力机制中，文本中所有的单词对(pi, pj)，无论其位置远近，均直接计算注意力函数值。这使得信息可以在相隔任意距离的单词间交互，大大提高了信息的传递效率。而且每个单词计算注意力向量的过程都是独立的，可以用并行计算提高运行速度。而RNN因为其线性结构导致其在信息传递过程中不能进行并行计算。

&emsp;&emsp;&emsp;&emsp;但是，自注意力机制完全舍弃了单词的位置信息，而单词的顺序和出现位置也会对其语义产生影响。因此，自注意力机制可以和RNN同时使用。此外，在后续会介绍位置编码的方法，为自注意力加入单词的位置信息。

## 上下文编码

&emsp;&emsp;&emsp;&emsp;在机器阅读理解的交互层中也常常使用编码层中的上下文编码技术。交互层可以交替使用互注意力、自注意力以及上下文编码。以文章部分为例，互注意力机制可以获得问题的信息，自注意力机制可以获得文章内部所有单词之间的信息，上下文编码则使用RNN基于文章单词的位置信息进行语义的传递。通过这些步骤的反复使用可以使模型更好地理解单词、短语、句子以及文章的语义信息，同时融入对问题的理解，从而提高预测答案的准确度。

&emsp;&emsp;&emsp;&emsp;由于交互层对于注意力和上下文编码的使用比较灵活，也使其机器阅读理解模型中发展最多样、成果最丰富的一部分。例如，2016年机器阅读理解BiDAF在交互层中使用了RNN——>互注意力——>RNN的结构；2017年的R-net模型在交互层中使用了RNN——>互注意力——>RNN——>自注意力——>RNN的结构；2018年的FusionNet模型在交互层中采用了互注意力——>RNN——>自注意力——>RNN的结构。

&emsp;&emsp;&emsp;&emsp;但是我们也注意到，随着交互层结构的复杂化，容易导致参数过多、模型过深，引起梯度消失、梯度爆炸、难以收敛或过拟合等不利于模型优化的现象。因此，一般建议可以从较少的层数开始，逐渐增加注意力和上下文编码的模块，并在验证数据上观察效果，同时配合使用Dropout，梯度裁剪（gradient clipping）等辅助手段加速优化进程。

# 输出层

&emsp;&emsp;&emsp;&emsp;输出层是机器阅读理解模型计算并输出答案的模块。经过编码层和交互层的计算，模型已经掌握了问题和文章的语义信息并进行了信息见的交互传递，具备了生成答案的条件。输出层的主要任务是，根据任务要求的方式生成答案，并构造合理的损失函数便于模型在训练数据集上进行优化。

## 构造问题的表示向量

&emsp;&emsp;&emsp;&emsp;在经过交互层的处理后，问题中的n个单词均得到向量表示，记作(q1, q2, ..., qn)。为了从文章中生成答案，通常将问题作为一个整体与文章中的单词进行匹配运算。因此，需要用一个向量q表示整个问题，以方便后续处理。此前介绍的由词向量生成文本向量的3种方法：RNN最终状态、CNN和池化、含参加权和。这些方法都可以从问题中所有单词的向量表示(q1, q2, ..., qn)生成q。

&emsp;&emsp;&emsp;&emsp;以含参加权和为例，参数为一个向量b，维度与qi相同。首先，使用内积操作给每个词向量计算一个分数si=b^T*qi。然后使用softmax操作将si归一化成和为1的权重(w1, w2, ..., wn)。最终，利用权重计算所有问题单词向量的加权和，得到一个代表问题的向量q。经过交互层的处理，向量q中已包含问题中所有单词的上下文信息，也包括了文章的信息。

## 多项选择式答案生成

&emsp;&emsp;&emsp;&emsp;在答案为多项选择类型的阅读理解数据集中，除了文章和问题外，还以自然语言的形式提供了若干选项。为了预测正确选线，模型需要在输出层对每个选项计算一个分数，最后选取分数最大的选项作为输出。

&emsp;&emsp;&emsp;&emsp;设一共有K个选项，可以用类似于处理问题的方法分析每个选项的语义：对选项中每个单词进行编码，再和问题及文章计算注意力向量，从而得到一个向量ck代表第k个选项的语义。然后，综合文章、问题与选项计算该选项的得分。这里可以比较灵活地设计各种计算选项得分的网络结构。在多个选项中选择正确答案是分类问题，属于自然语言理解的范畴，因此在优化时可以使用交叉熵作为损失函数。在整个计算过程中，模型不需要将网络复制K份，而是共用同一个网络计算每个选项的得分。

## 区间式答案生成

&emsp;&emsp;&emsp;&emsp;区间式答案是指答案由文章中一段连续的语句组成。对于一篇长度为m个词的文章，可能的区间式答案有m(m-1)/2种。对于区间型答案的机器阅读理解任务，模型的输出层应预测答案区间的开始位置和结束位置。这里输出层对所有单词计算两个得分，分别为第i个单词作为答案区间第一个词的可能性分数$g_i^S$以及其作为答案区间最后一个词的可能性分数$g_i^E$。由于预测答案的开始位置和结束位置均为多分类任务，因此可以采用交叉熵损失函数（两个多分类问题的交叉熵之和）。以下是答案区间生成的代码。

In [1]:
import torch
import numpy as np

In [24]:
"""
设文本共m个词，prob_s是大小为m的开始位置概率，prob_e是大小为m的结束位置概率，均为一维PyTorch张量。
L为答案区间可以包含的最大单词数。输出为概率最高的区间在文本中的开始位置和结束位置。
"""
def get_best_interval(prob_s, prob_e, L):
    prob = torch.ger(prob_s, prob_e)  # 获得m*m的矩阵，其中prob[i,j]=prob_s[i]*prob_e[j]
    # 将prob限定为上三角矩阵且只保留主对角线及其右上方L-1条对角线的值，其他值清零。即如果i>j或j-i+1>L，设置prob[i,j]*0
    prob.triu_().tril_(L - 1)
    prob = prob.numpy()  # 转为NumPy数组
    # 获得概率最高的答案区间，开始位置为第best_start个词，结束位置为第best_end个词
    best_start, best_end = np.unravel_index(np.argmax(prob), prob.shape)
    return best_start, best_end

In [21]:
# test

In [23]:
sent_len = 20
L = 5
prob_s = F.softmax(torch.randn(sent_len), dim=0)
prob_e = F.softmax(torch.randn(sent_len), dim=0)
best_start, best_end = get_best_interval(prob_s=prob_s, prob_e=prob_e, L=L)
print(best_start, best_end)

48


## 自由式答案生成

&emsp;&emsp;&emsp;&emsp;自由式答案是指答案可以为任何自然语言处理形式，不需要其中所有单词均来自文章。生成自由式答案的过程就是自然语言生成的过程。因此，对于这种任务，模型的输出层基本采用序列到序列模型，也就是编码器-解码器。

&emsp;&emsp;&emsp;&emsp;编码器从交互层获得文本中每个单词的向量(p1, p2, ..., pm)，然后使用双向RNN处理文本的所有单词，第i个单词产生输出状态为$h_i^{enc}$。这里可以使用问题向量q作为RNN的初始状态$h_0^{enc}$。

&emsp;&emsp;&emsp;&emsp;解码器使用单向RNN依次产生答案的单词。RNN初始状态$h_0^{dec}$为编码器最后一个状态$h_m^{enc}$。一般设置第一个答案单词为文本开始位置标识符“<s>”，并使用它的词表向量作为第一个RNN单元的输入$t_0$。这个单元的输出状态$h_1^{dec}$用于产生模型预测的第一个答案单词。

&emsp;&emsp;&emsp;&emsp;设词表的大小为|V|，词表向量维度是d，建立大小为d*|V|的全连接层将$h_1^{dec}$转化成一个|V|维向量，表示模型对词表中每个单词的打分。这些分数经过softmax可以得到预测概率$P_1, P_2, ..., P_{|V|}$。在训练时，如果标准答案的第一个单词是词表中的第i个单词，则此位置的损失函数值为$f_{cross_entropy}=-log(P_i)$。

&emsp;&emsp;&emsp;&emsp;接下来，将$h_1^{dec}$状态传递到第二个RNN单元。这个RNN单元的输入单词向量$t_1$取决于是否使用Teaching Forcing：如果使用Teaching Forcing，则使用标准答案的第一个单词；否则使用分数最高的单词，即模型预测的第一个单词。第二个RNN单元生成模型预测的第二个单词的概率，依次类推。

&emsp;&emsp;&emsp;&emsp;一般来说，编码器的词表，解码器的词表以及最终的全连接层共享其中的参数：大小均为d\*|V|或|V|\*d。这样做既可以减小参数个数，也可以大大提高训练效率和预测质量。

&emsp;&emsp;&emsp;&emsp;此外，词表一般采用特殊标识单词<UNK\>标识词表以外的词，所以解码器也可能在某些位置生成<UNK\>单词。一个提升准确度的技巧是，模型在最终输出答案时将所有生成答案里的<UNK\>用在文章中随机选择的单词替换，这样可以在去除<UNK\>的同时保证准确度不会下降。

### 注意力机制的应用

&emsp;&emsp;&emsp;&emsp;答案生成的各个位置的单词有可能与文章中不同的片段相关。但是，生成单词的解码器所获得的唯一与文章相关的信息是其初始状态，即编码器最后一个RNN单元的状态向量。但是，单个向量很难精确保留文章中所有片段的信息。因此，注意力机制常被应用在解码器中，在生成单词时提供文章的信息。下面介绍解码时注意力机制的使用方法。

&emsp;&emsp;&emsp;&emsp;为了减少 